In [76]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from gurobipy import GRB, Model

In [77]:
static = pd.read_csv('static_with_t.csv')
detected = pd.read_csv('randomized_graph.csv')

In [78]:
node_name = 'conn_k'

In [79]:
static_node = static[static['name'] == node_name].to_dict('records')[0]
static_node

{'name': 'conn_k',
 'number': 45,
 'x': 0.0,
 'y': -104.0,
 'z': -10.0,
 'parent': 'cross_k',
 'distance_to_parent': 56}

In [80]:
detected_node = detected[detected['name'] == node_name].to_dict('records')[0]
detected_node

{'name': 'conn_k',
 'number': 45,
 'x': -3.42,
 'y': -104.0,
 'z': -9.4,
 'parent': 'cross_k'}

In [81]:
detected_parent = detected[detected['name'] == static_node['parent']].to_dict('records')[0]
detected_parent


{'name': 'cross_k',
 'number': -1,
 'x': -12.22,
 'y': -50.0,
 'z': 4.45,
 'parent': 'cross_f'}

In [82]:
parent_x = detected_parent["x"]
parent_y = detected_parent["y"]
parent_z = detected_parent["z"]

detected_x = detected_node["x"]
detected_y = detected_node["y"]
detected_z = detected_node["z"]

distance_to_parent = static_node["distance_to_parent"]

In [83]:
quadratic_model = Model('quadratic')

In [84]:
x = quadratic_model.addVar(name = 'x', vtype=GRB.CONTINUOUS)
y = quadratic_model.addVar(name = 'y', vtype=GRB.CONTINUOUS)
z = quadratic_model.addVar(name = 'z', vtype=GRB.CONTINUOUS)

In [85]:
detected_x_rel = detected_x - parent_x
detected_y_rel = detected_y - parent_y
detected_z_rel = detected_z - parent_z

print(f"detected_x: {detected_x}, detected_y: {detected_y}, detected_z: {detected_z}")
print(f"parent_x: {parent_x}, parent_y: {parent_y}, parent_z: {parent_z}")
print(f"detected_x_rel: {detected_x_rel}, detected_y_rel: {detected_y_rel}, detected_z_rel: {detected_z_rel}")

detected_x: -3.42, detected_y: -104.0, detected_z: -9.4
parent_x: -12.22, parent_y: -50.0, parent_z: 4.45
detected_x_rel: 8.8, detected_y_rel: -54.0, detected_z_rel: -13.850000000000001


In [86]:
sign_matrix = np.sign(np.array([detected_x_rel, detected_y_rel, detected_z_rel]))
sign_matrix

array([ 1., -1., -1.])

In [87]:
detected_x_rel = np.abs(detected_x_rel)
detected_y_rel = np.abs(detected_y_rel)
detected_z_rel = np.abs(detected_z_rel)

In [88]:
quadratic_model.addQConstr(
    x**2 + y**2 + z**2 <= distance_to_parent**2)

<gurobi.QConstr Not Yet Added>

In [89]:
obj_fn = (x - detected_x_rel)**2 + (y - detected_y_rel)**2 + (z - detected_z_rel)**2

In [90]:
quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)

In [91]:
quadratic_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x70926744
Model has 3 quadratic objective terms
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+01, 1e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [3e+03, 3e+03]
Presolve time: 0.02s
Presolved: 8 rows, 9 columns, 12 nonzeros
Presolved model has 2 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.400e+01
 Factor NZ  : 3.600e+01
 Factor Ops : 2.040e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       P

In [92]:
calc_x = quadratic_model.getVarByName('x').x
calc_y = quadratic_model.getVarByName('y').x
calc_z = quadratic_model.getVarByName('z').x

In [93]:
calc_x = calc_x * sign_matrix[0]
calc_y = calc_y * sign_matrix[1]
calc_z = calc_z * sign_matrix[2]


In [94]:
#add rel
calc_x_f = calc_x + parent_x
calc_y_f = calc_y + parent_y
calc_z_f = calc_z + parent_z

In [95]:
print(f"calc_x_f: {calc_x_f}, calc_y_f: {calc_y_f}, calc_z_f: {calc_z_f}")
print(f"detected_x: {detected_x}, detected_y: {detected_y}, detected_z: {detected_z}")

calc_x_f: -3.488329697272931, calc_y_f: -103.58080574668253, calc_z_f: -9.292459310320204
detected_x: -3.42, detected_y: -104.0, detected_z: -9.4


In [96]:
print('Obj: %g' % np.sqrt(quadratic_model.objVal))

Obj: 0.43813


In [97]:
print(f"function value: {np.sqrt(quadratic_model.objVal)}")
print(f"Distance to detected node: {np.sqrt((calc_x-detected_x_rel)**2 + (calc_y-detected_y_rel)**2 + (calc_z-detected_z_rel)**2)}")
print(f"calculated distance to parent: {np.sqrt((calc_x_f-parent_x)**2 + (calc_y_f-parent_y)**2 + (calc_z_f-parent_z)**2)}")
print(f"Original distance to parent: {distance_to_parent}")


function value: 0.4381298648062617
Distance to detected node: 111.06294721843395
calculated distance to parent: 55.999999987813034
Original distance to parent: 56


In [98]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from transformation_utils import get_rotation, rotate_graph, get_rotation_nn, optim_rotation

G = nx.Graph()
G1 = nx.Graph()

G.add_node('static_node', pos=(static_node['x'], static_node['z'], static_node['y']))
G.add_node('detected_node', pos=(detected_node['x'], detected_node['z'], detected_node['y']))
G.add_node('detected_parent', pos=(detected_parent['x'], detected_parent['z'], detected_parent['y']))
G1.add_node('calculated_node', pos=(calc_x_f, calc_z_f, calc_y_f))

plot_graph(static_graph=G, detected_graph=G1, name='qcqp', save=True)




Graph saved to qcqp.html
